In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 47.3 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 25.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=5bc63c8a5cf09e3c7959f65ab79d3a8d5c4f85edc404b6e87dcf34f517df28c7
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 31.9 MB/s 
     |████████████████████████████████| 769 kB 26.3 MB/s 
     |████████████████████████████████| 3.0 MB 60.8 MB/s 
     |████████████████████████████████| 895 kB 61.1 MB/s 


In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-uch4yg48
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-uch4yg48
     |████████████████████████████████| 132 kB 23.7 MB/s 
     |████████████████████████████████| 4.5 MB 76.6 MB/s 
     |████████████████████████████████| 4.0 MB 63.1 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
     |████████████████████████████████| 596 kB 63.2 MB/s 
     |████████████████████████████████| 6.6 MB 56.5 MB/s 
     |████████████████████████████████| 8.7 MB 61.9 MB/s 
     |████████████████████████████████| 79 kB 11.0 MB/s 
     |████████████████████████████████| 127 kB 76.5 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=bdb852c70371d2659976c7d7cfd14ed2acf176485e4780b72ff2d36e93f15271
  Stored in directory: /tmp/pip-ephem-wheel-cache-o9xai526/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd

In [28]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

ModuleNotFoundError: ignored

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [27]:
#GPU 사용
device = torch.device("cuda:0")

NameError: ignored

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
len(chatbot_data)

38594

In [ ]:
chatbot_data.sample(n=10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
4990,참는다는게 진짜 힘들다..,공포,NaN,NaN,NaN,NaN,NaN
25997,상하이랑 쎄울하고 붙으면 5대떡 각이겠는데,중립,NaN,NaN,NaN,NaN,NaN
3091,전 직장에서 여자들이 많다보니까 스트레스도 많이 받고상처도 많이 받아서 더 두렵습니다..,공포,NaN,NaN,NaN,NaN,NaN
33904,이래서 골수 외교관리인 반기문이 믿음이 안간다. ㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN
32369,ㅋㅋㅋㅋㅋㅋㅋ 이번엔 감독탓 ㅋㅋㅋ,행복,NaN,NaN,NaN,NaN,NaN
34580,바보 기상청,혐오,NaN,NaN,NaN,NaN,NaN
17536,첫 아가랏 걱정이 되네요..,슬픔,NaN,NaN,NaN,NaN,NaN
19959,"이렇게 축구,야구,골프의 신들을 데려가네요 ㅠㅠ",슬픔,NaN,NaN,NaN,NaN,NaN
37231,헌재는 침몰하는 나라를 구한다는 비상한 신념과 의지로 박근혜 탄핵 인용하시길~,혐오,NaN,NaN,NaN,NaN,NaN
2281,이런 일은 처음이라 여자로써 수치스럽고 추후에 제건강에 대해 걱정되고 난임가능성이 ...,공포,NaN,NaN,NaN,NaN,NaN


In [ ]:
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [ ]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['최악의 상황중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

28945
9649


In [8]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [9]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [10]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 1189,  517, 6188, 7245, 7063,  517,  463, 3486, 7836, 5966,
        1698,  517, 6188, 7245, 7063,  517,  463, 1281, 7870, 1801, 6885,
        7088, 5966, 1698, 5837, 5837,  517, 6188, 7245, 6398, 6037, 7063,
         517,  463,  517,  463,  517,  364,  517,  364,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(42, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 5)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [11]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

NameError: ignored

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

NameError: ignored

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

NameError: ignored

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/453 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.0615413188934326 train acc 0.15625
epoch 1 batch id 201 loss 1.2674336433410645 train acc 0.2964863184079602
epoch 1 batch id 401 loss 1.2778666019439697 train acc 0.3847023067331671
epoch 1 train acc 0.39751128100246724


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.5501820347344235


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.3950776159763336 train acc 0.84375
epoch 5 batch id 201 loss 0.5071197152137756 train acc 0.7879353233830846
epoch 5 batch id 401 loss 0.46097230911254883 train acc 0.7978101620947631
epoch 5 train acc 0.8014705882352942


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 5 test acc 0.5556029531017705


In [ ]:
## 학습 모델 저장
for i in tqdm(range(1)):
  PATH = '/content/drive/MyDrive/colab/' # google 드라이브 연동 해야함. 관련코드는 뺐음
  torch.save(model, PATH + 'KoBERT_model.pt')  # 전체 모델 저장
  torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
  torch.save({
      'model': model.state_dict(),
      'optimizer': optimizer.state_dict()
  }, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

100%|██████████| 1/1 [00:08<00:00,  8.13s/it]


In [12]:
## 학습 모델 로드
PATH = '/content/drive/MyDrive/colab/'
model = torch.load(PATH + 'KoBERT_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

<All keys matched successfully>

In [15]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [25]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_ㅇlength= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
                cnt_list.append("공포")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
                cnt_list.append("놀람")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
                cnt_list.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
                cnt_list.append("슬픔")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
                cnt_list.append("중립")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
                cnt_list.append("행복")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")
                cnt_list.append("혐오")
      
            print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
# 한글 형태소 분석기 설치
!pip install konlpy
!pip install pandas

import pandas as pd

data = pd.read_excel("/content/drive/MyDrive/국내노래 (2).xlsx")

print(len(data))


     |████████████████████████████████| 19.4 MB 4.2 MB/s 
     |████████████████████████████████| 448 kB 49.9 MB/s 
10


In [ ]:
# [^ㄱ-ㅎㅏ-ㅣ가-힣 ] : 한글과 빈공백을 표시하는 정규식 (마지막에 빈 공백이 들어가 있음 = 띄어쓰기)
# ^ : 부정 
# [^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ] : 한글,영문,빈공백을 포함
# [^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9 ] : 한글,영문,숫자,빈공백을 포함
data["가사"] = data["가사"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [ ]:
import konlpy
from konlpy.tag import Okt    # 한글 형태소 분석기 (Kkm, Hannaum, Mecab, Twitter ...)
from tqdm import tqdm

okt = Okt()

X_train =[]

stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# tqdm() : 반복문의 진행율을 보기 위한 것
for sen in tqdm(data["가사"]) :
   temp_X = []

  # 문장을 형태소로 분리하여 저장
   temp_X = okt.morphs(sen)

  # 불용어 처리
  # word가 stopwords에 없는 경우만 저장
   temp_X = [word for word in temp_X if word not in stopwords]

   X_train.append(temp_X)

100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


In [ ]:
import pickle  # 파이썬에서 사용하는 데이터 타입을 그대로 저장

with open("/content/drive/MyDrive/X_train2.pickle","wb") as f : 
  pickle.dump(X_train, f)

In [ ]:
# 저장된 파일을 읽어오기
import pickle 

X_train2 = []

with open("/content/drive/MyDrive/colab/X_train2.pickle","rb") as f :
  X_train2 = pickle.load(f)

In [1]:
import pandas as pd
X_train3 = pd.read_excel('/content/drive/MyDrive/국내노래 (사랑).xlsx')

FileNotFoundError: ignored

In [176]:
y=X_train3.iloc[:,2]

In [138]:
X_train3['가사']

0    [참 많이 궁금해, 전부 다 궁금해, 왜 잠이 안 오고 네 얼굴만 보여, , 나도 ...
1    [커피를 너무 많이 마셨나 봐요 , 심장이 막 두근대고 잠은 잘 수가 없어요, 한참...
2    [달이 차고 내 마음도 차고, 이대로 담아 두기엔, 너무 안타까워, 너를 향해 가는...
3    [너무 걱정 하지는 마, 보란듯이 살아볼거야, 후회는 사치일 뿐이야, 다시 시작해 ...
4    [이윽고 내가 한눈에 너를 알아봤을 때, 모든 건 분명 달라지고 있었어 , 내 세상...
5    [표현이 서툰 것도 잘못인가요? , 나 차가운 도시에 따뜻한 여잔데, 그냥 좋아한단...
6    [밤하늘의 별을 따서 너에게 줄래 너는 내가 사랑하니까 더 소중하니까, 오직 너 아...
7    [느낌이 오잖아 떨리고 있잖아 , 언제까지 눈치만 볼 거니, 네 맘을 말해봐 딴청 ...
8    [너를 만난 그 이후로, 사소한 변화들에 행복해져, 눈이 부시게 빛 나는 아침, 너...
9    [너와 함께 하고 싶은 일들을 상상하는 게, 요즘 내 일상이 되고, 너의 즐거워하는...
Name: 가사, dtype: object

In [16]:
import pandas as pd
X_train3 = pd.read_excel('/content/drive/MyDrive/국내노래.xlsx')
y = X_train3.iloc[:,2]
nt_list=[]
cnt = 0
for i in  y:
    text = y[cnt]
    text_r2=text.split('\n')
    y[cnt]=text_r2
    nt_list.append(y[cnt])
    cnt = cnt +1 

In [21]:
item={"노래분할":nt_list}
df=pd.DataFrame(item)
df.to_csv("노래분할.csv",encoding='utf-8-sig')

In [23]:
nt_list[0][0]

'참 많이 궁금해'

In [212]:
mtp_list

[Counter({'공포': 15, '놀람': 9, '슬픔': 26, '중립': 14, '행복': 28, '혐오': 2}),
 Counter({'공포': 6, '놀람': 5, '분노': 1, '슬픔': 9, '중립': 7, '행복': 28, '혐오': 1}),
 Counter({'공포': 4, '놀람': 7, '슬픔': 10, '중립': 7, '행복': 18, '혐오': 1}),
 Counter({'공포': 4, '놀람': 2, '슬픔': 15, '중립': 5, '행복': 10, '혐오': 3}),
 Counter({'공포': 1, '놀람': 1, '분노': 2, '슬픔': 11, '중립': 1, '행복': 11, '혐오': 3}),
 Counter({'공포': 6, '놀람': 1, '분노': 2, '슬픔': 13, '중립': 9, '행복': 15, '혐오': 2}),
 Counter({'공포': 1, '슬픔': 20, '중립': 1, '행복': 7}),
 Counter({'공포': 6, '놀람': 2, '슬픔': 5, '중립': 8, '행복': 16, '혐오': 7}),
 Counter({'공포': 4, '놀람': 2, '슬픔': 18, '중립': 5, '행복': 12, '혐오': 1}),
 Counter({'공포': 5, '슬픔': 8, '중립': 5, '행복': 11}),
 Counter({'공포': 15, '놀람': 12, '분노': 3, '슬픔': 12, '중립': 29, '행복': 16, '혐오': 2}),
 Counter({'슬픔': 7, '중립': 9, '행복': 15, '혐오': 2}),
 Counter({'공포': 4, '놀람': 1, '분노': 3, '슬픔': 11, '중립': 15, '행복': 2}),
 Counter({'공포': 3, '슬픔': 4, '중립': 6, '행복': 8, '혐오': 5}),
 Counter({'공포': 5, '놀람': 5, '슬픔': 21, '중립': 7, '행복': 10}),
 Counter({'공포': 2,

In [209]:
mtp_list

[Counter({'공포': 15, '놀람': 9, '슬픔': 26, '중립': 14, '행복': 28, '혐오': 2}),
 Counter({'공포': 6, '놀람': 5, '분노': 1, '슬픔': 9, '중립': 7, '행복': 28, '혐오': 1}),
 Counter({'공포': 4, '놀람': 7, '슬픔': 10, '중립': 7, '행복': 18, '혐오': 1}),
 Counter({'공포': 4, '놀람': 2, '슬픔': 15, '중립': 5, '행복': 10, '혐오': 3}),
 Counter({'공포': 1, '놀람': 1, '분노': 2, '슬픔': 11, '중립': 1, '행복': 11, '혐오': 3}),
 Counter({'공포': 6, '놀람': 1, '분노': 2, '슬픔': 13, '중립': 9, '행복': 15, '혐오': 2}),
 Counter({'공포': 1, '슬픔': 20, '중립': 1, '행복': 7}),
 Counter({'공포': 6, '놀람': 2, '슬픔': 5, '중립': 8, '행복': 16, '혐오': 7}),
 Counter({'공포': 4, '놀람': 2, '슬픔': 18, '중립': 5, '행복': 12, '혐오': 1}),
 Counter({'공포': 5, '슬픔': 8, '중립': 5, '행복': 11})]

In [211]:
mtp_list

[Counter({'공포': 15, '놀람': 9, '슬픔': 26, '중립': 14, '행복': 28, '혐오': 2}),
 Counter({'공포': 6, '놀람': 5, '분노': 1, '슬픔': 9, '중립': 7, '행복': 28, '혐오': 1}),
 Counter({'공포': 4, '놀람': 7, '슬픔': 10, '중립': 7, '행복': 18, '혐오': 1}),
 Counter({'공포': 4, '놀람': 2, '슬픔': 15, '중립': 5, '행복': 10, '혐오': 3}),
 Counter({'공포': 1, '놀람': 1, '분노': 2, '슬픔': 11, '중립': 1, '행복': 11, '혐오': 3}),
 Counter({'공포': 6, '놀람': 1, '분노': 2, '슬픔': 13, '중립': 9, '행복': 15, '혐오': 2}),
 Counter({'공포': 1, '슬픔': 20, '중립': 1, '행복': 7}),
 Counter({'공포': 6, '놀람': 2, '슬픔': 5, '중립': 8, '행복': 16, '혐오': 7}),
 Counter({'공포': 4, '놀람': 2, '슬픔': 18, '중립': 5, '행복': 12, '혐오': 1}),
 Counter({'공포': 5, '슬픔': 8, '중립': 5, '행복': 11}),
 Counter({'공포': 15, '놀람': 12, '분노': 3, '슬픔': 12, '중립': 29, '행복': 16, '혐오': 2}),
 Counter({'슬픔': 7, '중립': 9, '행복': 15, '혐오': 2}),
 Counter({'공포': 4, '놀람': 1, '분노': 3, '슬픔': 11, '중립': 15, '행복': 2}),
 Counter({'공포': 3, '슬픔': 4, '중립': 6, '행복': 8, '혐오': 5}),
 Counter({'공포': 5, '놀람': 5, '슬픔': 21, '중립': 7, '행복': 10}),
 Counter({'공포': 2,

In [210]:
from collections import Counter
X_train3 = pd.read_excel('/content/drive/MyDrive/국내노래.xlsx')
y=X_train3.iloc[:,2]

cnt_list=[]
total=[]
mtp_list=[]
cnt = 0

for i in  y:
    text = y[cnt]
    text_r2=text.split('\n')
    y[cnt]=text_r2
    cnt = cnt +1 

cnt2 = 0
cnt3 = 0

for j in range(len(y)):
  try:
    for k in range(100):
      predict(y[cnt2][cnt3])
      cnt3=cnt3+1
      total.append(cnt_list)
      
  except:pass
  emotion = Counter(cnt_list)
  mtp_list.append(emotion)
  cnt_list=[]
  cnt2= cnt2+1
  cnt3 = 0
  

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 혐오가 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 중립이 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 중립이 느껴집니다.
>> 입력하신 내용에서 중립이 느껴집니다.
>> 입력하신 내용에서 놀람이 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 중립이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 행복이

In [ ]:
from collections import Counter
mt_list=[]
mtp_list=[]
m_list=[]
cnt = 0
cnt2 = 0
cnt3 = 0
cnt_list=[]
total=[]

for i in X_train2 :
    text = X_train2[cnt]
    text_r=",".join(text)
    text_r2=text_r.split(',')
    try:
      for j in range(len(text_r2)):
          m_list.append(text_r2[cnt2]+text_r2[cnt2+1]+text_r2[cnt2+2]+text_r2[cnt2+3])
          cnt2=cnt2+2
    except:pass
    mt_list.append(m_list)
    
    for k in range(len(m_list)):
        list=mt_list[cnt][cnt3]
        predict(list)
        cnt3=cnt3+1
    total.append(cnt_list)
    
    emotion = Counter(cnt_list)
    mtp_list.append(emotion)
    
    cnt_list=[]
    m_list=[]
    cnt=cnt+1
    cnt2=0
    cnt3=0
    

In [ ]:
m_list=[]
cnt2 = 0

text = X_train2[0]
text_r=",".join(text)
text_r2=text_r.split(',')
for j in range(len(text_r2)):
        m_list.append(text_r2[cnt2])
        cnt2=cnt2+1

In [ ]:
try:
    for j in range(len(text_r2)):
        m_list.append(text_r2[cnt2]+text_r2[cnt2+1])
        cnt2=cnt2+1
except:pass

In [ ]:
m_list[0]

'너함께'

In [ ]:
from collections import Counter
mt_list=[]
mtp_list=[]
m_list=[]
cnt = 0
cnt2 = 0
cnt3 = 0
cnt_list=[]
total=[]

for i in X_train2 :
    text = X_train2[cnt]
    text_r=",".join(text)
    text_r2=text_r.split(',')
    try:
      for j in range(len(text_r2)):
          m_list.append(text_r2[cnt2]+text_r2[cnt2+1]+text_r2[cnt2+2]+text_r2[cnt2+3])
          cnt2=cnt2+2
    except:pass
    mt_list.append(m_list)
    
    for k in range(len(m_list)):
        list=mt_list[cnt][cnt3]
        predict(list)
        cnt3=cnt3+1
    total.append(cnt_list)
    
    emotion = Counter(cnt_list)
    mtp_list.append(emotion)
    
    cnt_list=[]
    m_list=[]
    cnt=cnt+1
    cnt2=0
    cnt3=0
    

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 혐오가 느껴집니다.
>> 입력하신 내용에서 중립이 느껴집니다.
>> 입력하신 내용에서 혐오가 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 중립이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 혐오가 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 혐오가 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 공포가 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 놀람이 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 놀람이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 분노가 느껴집니다.
>> 입력하신 내용에서 분노가 느껴집니다.
>> 입력하신 내용에서 중립이 느껴집니다.
>> 입력하신 내용에서 중립이 느껴집니다.
>> 입력하신 내용에서 혐오가 느껴집니다.
>> 입력하신 내용에서 행복이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이 느껴집니다.
>> 입력하신 내용에서 슬픔이

In [ ]:
pre=predict("참")

NameError: ignored

In [ ]:
text = X_train2[0]
text_r=",".join(text)

In [ ]:
text_r2=text_r.split(',')

In [ ]:
mt_list[1]

['커피너무많이마셨나',
 '봐요심장막두',
 '근대고잠수가',
 '없어요한참뒤별빛',
 '내리면난다시잠들',
 '순없겠죠지나간새벽',
 '을다새면다시',
 '네곁잠들겠죠너',
 '품잠든난마치',
 '천사된것만',
 '같아요난그대',
 '품별빛을쏟아',
 '내리고은하수만들어어디',
 '든날아가게할거야줄',
 '게내네곁',
 '저별을따',
 '네게만줄',
 '게내리는비처럼반짝이는',
 '널가지고싶어',
 '엄지검지만해도',
 '내마음을너무',
 '표현해붙어안달',
 '나니까마냥떨리기만게',
 '아냐준비되면쏘아',
 '올린인공위성처럼네',
 '주윌마구맴돌려해더',
 '가까워진다면네가져줄래떨림을',
 '어제내기분참',
 '좋아서지나간행성에다가',
 '그대이름새겨놓았죠',
 '한참뒤별빛내리면',
 '그별가장밝게',
 '빛나요지나간새벽을',
 '다새면다시네',
 '곁잠들겠죠별빛아래',
 '잠든난마치온',
 '우주가진것만',
 '같아난그대품별빛',
 '을쏟아내리고은하수',
 '만들어어디든날아가게',
 '할거야줄게내',
 '네곁저별',
 '을따네게',
 '만줄게줄',
 '게내줄게',
 '라라라라라라라라라라']

In [ ]:
mtp_list # 단어 10개 앞뒤 o 

[Counter({0: 6, 1: 2, 2: 3, 3: 24, 4: 5, 5: 13, 6: 8}),
 Counter({0: 7, 1: 1, 2: 2, 3: 11, 5: 7, 6: 7}),
 Counter({0: 3, 1: 3, 2: 2, 3: 10, 4: 3, 5: 10, 6: 1}),
 Counter({2: 1, 3: 18, 5: 4, 6: 5}),
 Counter({0: 2, 1: 1, 2: 3, 3: 12, 4: 1, 5: 4, 6: 4}),
 Counter({0: 4, 2: 6, 3: 22, 4: 4, 5: 6, 6: 9}),
 Counter({0: 1, 2: 1, 3: 31, 4: 3, 5: 2, 6: 3}),
 Counter({0: 2, 1: 6, 2: 6, 3: 6, 4: 1, 6: 19}),
 Counter({2: 1, 3: 21, 4: 1, 5: 8, 6: 4}),
 Counter({0: 1, 3: 10, 4: 2, 5: 2, 6: 6})]

In [ ]:
mtp_list # 단어 10개 앞뒤 x 

[Counter({0: 1, 1: 2, 2: 2, 3: 14, 4: 1, 5: 7, 6: 4}),
 Counter({0: 4, 3: 6, 5: 4, 6: 4}),
 Counter({0: 2, 1: 2, 2: 1, 3: 5, 4: 2, 5: 4}),
 Counter({2: 1, 3: 10, 5: 1, 6: 2}),
 Counter({0: 1, 1: 1, 2: 2, 3: 5, 4: 1, 5: 2, 6: 2}),
 Counter({0: 2, 2: 5, 3: 12, 4: 1, 5: 3, 6: 3}),
 Counter({0: 1, 3: 15, 4: 2, 5: 1, 6: 2}),
 Counter({0: 1, 1: 4, 2: 3, 3: 3, 4: 1, 6: 8}),
 Counter({2: 1, 3: 12, 5: 4, 6: 1}),
 Counter({0: 1, 3: 5, 5: 1, 6: 4})]

In [ ]:
mtp_list # 단어 4개 앞뒤 x 

[Counter({0: 7, 1: 11, 2: 8, 3: 20, 4: 5, 5: 12, 6: 14}),
 Counter({0: 5, 1: 5, 2: 10, 3: 9, 4: 1, 5: 5, 6: 11}),
 Counter({0: 2, 1: 3, 2: 2, 3: 11, 4: 11, 5: 8, 6: 4}),
 Counter({0: 1, 1: 1, 2: 3, 3: 14, 4: 1, 5: 6, 6: 11}),
 Counter({0: 4, 1: 6, 2: 1, 3: 11, 5: 3, 6: 11}),
 Counter({0: 8, 1: 4, 2: 6, 3: 8, 4: 15, 5: 10, 6: 14}),
 Counter({0: 5, 1: 2, 2: 10, 3: 24, 4: 1, 5: 7, 6: 4}),
 Counter({0: 10, 1: 5, 2: 6, 3: 7, 4: 7, 5: 3, 6: 13}),
 Counter({0: 2, 2: 1, 3: 21, 4: 2, 5: 10, 6: 9}),
 Counter({0: 6, 1: 3, 2: 3, 3: 7, 4: 3, 5: 1, 6: 5})]

In [ ]:
mtp_list # 단어 4개 앞뒤 o

In [ ]:
mtp_list #단어 2개

[Counter({0: 38, 1: 79, 2: 30, 3: 38, 4: 38, 5: 56, 6: 30}),
 Counter({0: 22, 1: 31, 2: 19, 3: 29, 4: 19, 5: 32, 6: 31}),
 Counter({0: 10, 1: 34, 2: 7, 3: 25, 4: 41, 5: 36, 6: 11}),
 Counter({0: 4, 1: 15, 2: 15, 3: 45, 4: 19, 5: 31, 6: 19}),
 Counter({0: 15, 1: 31, 2: 12, 3: 26, 4: 19, 5: 16, 6: 24}),
 Counter({0: 21, 1: 53, 2: 22, 3: 33, 4: 65, 5: 46, 6: 19}),
 Counter({0: 23, 1: 37, 2: 17, 3: 70, 4: 33, 5: 23, 6: 10}),
 Counter({0: 18, 1: 45, 2: 25, 3: 16, 4: 58, 5: 14, 6: 28}),
 Counter({0: 15, 1: 18, 2: 8, 3: 60, 4: 27, 5: 39, 6: 13}),
 Counter({0: 18, 1: 23, 2: 6, 3: 16, 4: 24, 5: 17, 6: 8})]

In [ ]:
mtp_list # 단어 1개

[Counter({0: 57, 1: 103, 2: 10, 3: 50, 4: 31, 5: 47, 6: 12}),
 Counter({0: 28, 1: 69, 2: 4, 3: 26, 4: 23, 5: 29, 6: 5}),
 Counter({0: 10, 1: 72, 2: 3, 3: 27, 4: 15, 5: 38}),
 Counter({0: 12, 1: 43, 2: 7, 3: 31, 4: 18, 5: 32, 6: 6}),
 Counter({0: 18, 1: 53, 2: 5, 3: 21, 4: 17, 5: 21, 6: 9}),
 Counter({0: 29, 1: 89, 2: 6, 3: 39, 4: 31, 5: 55, 6: 11}),
 Counter({0: 63, 1: 56, 2: 2, 3: 42, 4: 17, 5: 29, 6: 5}),
 Counter({0: 27, 1: 59, 2: 16, 3: 34, 4: 33, 5: 26, 6: 10}),
 Counter({0: 26, 1: 38, 2: 3, 3: 46, 4: 22, 5: 40, 6: 6}),
 Counter({0: 13, 1: 38, 2: 3, 3: 16, 4: 15, 5: 26, 6: 2})]

In [26]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :\
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : Cause I'm a pilot anywhere 


NameError: ignored

In [24]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = text_r2[0]
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

NameError: ignored

In [ ]:
predict("불리하지만 어머 깜짝이야")

In [ ]:
!text_r2